In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
covid_confirmed_url = "https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv"
covid_deaths_url ="https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_deaths_usafacts.csv" 
covid_county_population_url = "https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv"
us_state_vaccinations_url="https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv"

In [ ]:
confirmed_cases = pd.read_csv(covid_confirmed_url)
covid_deaths = pd.read_csv(covid_deaths_url)
covid_county_population = pd.read_csv(covid_county_population_url)
us_state_vaccinations = pd.read_csv(us_state_vaccinations_url) 

In [ ]:
# US State abbriviation and Full names of states to ease the data merging.
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York State': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

In [ ]:
us_state_vaccinations_data=us_state_vaccinations
us_state_vaccinations_data= us_state_vaccinations_data.fillna(method='ffill')
us_state_vaccinations_data.head()

In [ ]:
def cleanPopulationData(df_to_clean):
    df_clean_filter_cond = df_to_clean[df_to_clean["population"]==0].index
    cleaned_df = df_to_clean.drop(df_clean_filter_cond)
    return cleaned_df

def cleanUsefactsDataFrame(df_raw):
    df_filter_cond = df_raw[df_raw["countyFIPS"]==0].index
    cleaned_df = df_raw.drop(df_filter_cond).reset_index(drop=True)
    cleaned_df = cleaned_df.groupby(['State']).sum()
    cleaned_df = cleaned_df.drop(['countyFIPS','StateFIPS'],axis=1)
    return cleaned_df


In [ ]:
# dataframes contains all population information with fullstates name: covid_county_population_data(County population)
county_population =cleanPopulationData(covid_county_population)
county_population['State_Names'] = county_population['State'].map(abbrev_us_state)
county_population
state_population = county_population.groupby(['State_Names']).sum().reset_index()
state_population = state_population.drop(['countyFIPS'],axis=1)
state_population
US_total_Poulation=state_population['population'].sum()
US_total_Poulation

#### Question 2. 
<h4>The vaccination data is reported by the entire county, state, US territory, and various groups
like the department of defense, long term care and others. Presumably the US data includes the state data and the special group data but not the US territory data. Is the US territory data included in the US totals. Justify your answer. 
        
    *US_data has DC, states and territories included, however groups are not.
    *Tried calculating Total_vaccination and total_dstributed column in order to cross verify.
    *After calculating multilple combinations of states, groups, DC and territories, have got simmiler results which shows
    *US_Data Inclues States+Territories+DC data not groups(only based upon the calculation).

In [ ]:
# lastest updated record for US and all other groups and states to compare updtaed record.
US_data= us_state_vaccinations_data.groupby('location').last().reset_index()
US_data.head()

In [ ]:
# Lists :state_names_list , terretories_list
filter_cond = county_population[county_population['State_Names']=="District of Columbia"].index
state_names_list=county_population.drop(filter_cond)
state_names_list=state_names_list['State_Names'].unique().tolist()

terretories_list =['American Samoa','Guam','Northern Mariana Islands','Puerto Rico','Virgin Islands','Marshall Islands','Federated States of Micronesia','Republic of Palau']

In [ ]:
vaccinations_data=US_data
# Dataframe With All state vaccination Records.
US_states_df= vaccinations_data[vaccinations_data['location'].isin(state_names_list)].reset_index(drop=True)

# Dataframe With All teritories vaccination Records.
US_territories_df=vaccinations_data[vaccinations_data['location'].isin(terretories_list)].reset_index(drop=True)

# Dataframe With All special group vaccination Records.
US_special_groups_df=vaccinations_data[~vaccinations_data['location'].isin(state_names_list)].reset_index(drop=True)
US_special_groups_df=US_special_groups_df[~US_special_groups_df['location'].isin(terretories_list)].reset_index(drop=True)
US_special_groups_df=US_special_groups_df[~US_special_groups_df['location'].isin(['District of Columbia','United States'])].reset_index(drop=True)

# Dataframe With United_states Records.
US_total_df=vaccinations_data[vaccinations_data['location'].isin(['United States'])].reset_index(drop=True)

# Dataframe With DC vaccination Records.
DC_df=vaccinations_data[vaccinations_data['location'].isin(['District of Columbia'])].reset_index(drop=True)
# US_states_df
# US_territories_df
# US_special_groups_df
# US_total_df
# DC_df

In [ ]:
# Combination 1
US_total=US_total_df['total_vaccinations'].sum()
US_states=US_states_df['total_vaccinations'].sum()
US_groups=US_special_groups_df['total_vaccinations'].sum()
DC_data=DC_df['total_vaccinations'].sum()
US_territory_data= US_territories_df['total_vaccinations'].sum()
print("US_TOTAL:{}" .format(US_total))

In [ ]:
all_included = US_states + US_groups + US_territory_data + DC_data
all_excluding_territories = US_states + US_groups + DC_data
all_excluding_DC = US_states + US_groups + US_territory_data
all_excluding_territories_DC = US_states + US_groups
all_excluding_groups = US_states + US_territory_data + DC_data
all_excluding_groups_DC = US_states + US_territory_data

print("Data includind all: {}".format(all_included)) # (more than us_total)
print("Data excluding territories: {}".format(all_excluding_territories)) # (more than us_total)
print("Data excluding DC: {}" .format(all_excluding_DC)) # (more)
print("Data excluding territories and DC: {}" .format(all_excluding_territories_DC))
print("Data excluding groups: {}" .format(all_excluding_groups)) # (very close to us_total)
print("Data excluding groups and DC: {}" .format(all_excluding_groups_DC)) # (very close)

In [ ]:
# Combination 2
US_total=US_total_df['total_distributed'].sum()
US_states=US_states_df['total_distributed'].sum()
US_groups=US_special_groups_df['total_distributed'].sum()
DC_data=DC_df['total_distributed'].sum()
US_territory_data= US_territories_df['total_distributed'].sum()
US_total

In [ ]:
all_included = US_states + US_groups + US_territory_data + DC_data
all_excluding_territories = US_states + US_groups + DC_data
all_excluding_DC = US_states + US_groups + US_territory_data
all_excluding_territories_DC = US_states + US_groups
all_excluding_groups = US_states + US_territory_data + DC_data
all_excluding_groups_DC = US_states + US_territory_data
print("Data includind all: {}".format(all_included)) # (more than us_total)
print("Data excluding territories: {}".format(all_excluding_territories)) # (more than us_total)
print("Data excluding DC: {}" .format(all_excluding_DC)) # (more than us_total)
print("Data excluding territories and DC: {}" .format(all_excluding_territories_DC))
print("Data excluding groups: {}" .format(all_excluding_groups)) # (exactly equal to US_totals)
print("Data excluding groups and DC: {}" .format(all_excluding_groups_DC)) # (very close)

#### Question 3. 
<h4>We would like to compute the percent of the population in each state that has been fully
vaccinated. Presumably the data reported got the various groups is not included in the individual state data. If we distributed the number of people fully vaccinated in those groups to
the states proportionally by population how much would that change the percent of people vaccinated in each state?

In [ ]:
def vaccinationPercent():
    vaccine_percentage_df = US_data.filter(['location','date','people_fully_vaccinated'], axis=1)
    vaccine_percentage_df = vaccine_percentage_df.merge(state_population, left_on='location', right_on='State_Names').reindex(columns=['location', 'date', 'people_fully_vaccinated', 'population'])
    vaccine_percentage_df ['people_vaccinated%']=(vaccine_percentage_df['people_fully_vaccinated']/vaccine_percentage_df['population'])*100
    vaccine_percentage_df.drop(vaccine_percentage_df[vaccine_percentage_df['location'] =="District of Columbia"].index, inplace=True)
    return vaccine_percentage_df.reset_index(drop=True)

In [ ]:
#Fully vaccinated total record of all special groups.
groups_fully_vaccinated=US_special_groups_df.filter(['location','date','people_fully_vaccinated'], axis=1)
groups_fully_vaccinated_data_total=groups_fully_vaccinated['people_fully_vaccinated'].sum() 
groups_fully_vaccinated_data_total

In [ ]:
#Distributing the Groups data Praportionaly to states and calculating Percentage of revised fully vaccinated data.
increased_percentage=vaccinationPercent()
increased_percentage['proportion']=increased_percentage['population']/US_total_Poulation
increased_percentage['distributed_numbers']=increased_percentage['proportion']*groups_fully_vaccinated_data_total
increased_percentage['fully_vaccinated_revised']=increased_percentage['people_fully_vaccinated']+increased_percentage['distributed_numbers']
increased_percentage['people_vaccinated%_revised']=(increased_percentage['fully_vaccinated_revised']/increased_percentage['population'])*100
increased_percentage['increased%']=increased_percentage['people_vaccinated%_revised']-increased_percentage['people_vaccinated%']
increased_percentage

#### Question 4
<h4>Produce a chart or table showing the percent of people in each state that have been fully
vaccinated. Sort the data by the percent of people vaccinated. You notebook should download the dataset so when it is run we get the most recent data.

In [ ]:
sorted_vaccination_per_state=vaccinationPercent()
sorted_vaccination_per_state=sorted_vaccination_per_state.sort_values(by=['people_vaccinated%']).reset_index(drop=True)
sorted_vaccination_per_state

#### Question 5
<h4>We want to see the progression from the start of the pandemic to now the percent of people that have some immunity in the US. Produce a plot of weekly data that contains three
lines. One showing the the percent of the living people have have or had covid. One showing the the percent of people that have had at least one covid vaccine shot. The third line
showing the sum of the two to estimate the number of people that have some immunity.
Over time sum will be an over estimate as people who had covid get a vaccine shot. As in
#4 the notebook should download the data to up-to-date results

In [ ]:
confirmed_cases_data=confirmed_cases
new_covid_cases= cleanUsefactsDataFrame(confirmed_cases_data)
new_covid_cases.head()

In [ ]:
covid_deaths_data=covid_deaths
new_covid_deaths=cleanUsefactsDataFrame(covid_deaths_data)
new_covid_deaths.head()

In [ ]:
# People who have had covid and alive
def immunityCountCal():
    people_have_had_covid=new_covid_cases.subtract(new_covid_deaths)
    people_have_had_covid.reset_index(inplace=True)
    people_have_had_covid = people_have_had_covid.melt(id_vars=['State'],value_name='Had Covid',var_name='date')
    return people_have_had_covid

In [ ]:
people_have_had_covid_alive = immunityCountCal()
people_have_had_covid_alive = people_have_had_covid_alive.groupby(['date']).sum().reset_index()
people_have_had_covid_alive.head()

In [ ]:
people_with_one_vaccine_dose = us_state_vaccinations_data.filter(['date','location','people_vaccinated'],axis=1)
people_with_one_vaccine_dose = people_with_one_vaccine_dose[people_with_one_vaccine_dose['location'].isin(state_names_list)].reset_index(drop=True)
people_with_one_vaccine_dose = people_with_one_vaccine_dose.groupby('date').sum().reset_index()
people_with_one_vaccine_dose.head()

In [ ]:
# To get the data of people who had covid and alive and number of people got at least one dose for each date. 
people_immunity_info = people_have_had_covid_alive.merge(people_with_one_vaccine_dose, how='left', on=['date']).fillna(0).reset_index(drop=True)
people_immunity_info['date'] = pd.to_datetime(people_immunity_info['date'])
people_immunity_info
# To get the data of people who had covid and alive and number of people got at least one dose per week. 
people_immunity_info = people_immunity_info.resample('W-mon', label='right', closed = 'right', on='date').apply(lambda x : x.iloc[-1])
people_immunity_info = people_immunity_info.reset_index(drop=True)
people_immunity_info

In [ ]:
# Percentage of people have some immunity in US per week
people_immunity_percent_info=pd.DataFrame()
people_immunity_percent_info['WeekDate']=people_immunity_info['date']
people_immunity_percent_info['% Had covid'] = (people_immunity_info['Had Covid']/US_total_Poulation)*100
people_immunity_percent_info['% Vaccinated'] =(people_immunity_info['people_vaccinated']/US_total_Poulation)*100
people_immunity_percent_info['% Immune'] = ((people_immunity_info['Had Covid']+people_immunity_info['people_vaccinated'])/US_total_Poulation)*100
people_immunity_percent_info['WeekDate']=people_immunity_percent_info['WeekDate'].map(lambda tf : tf.strftime("%d-%m-%Y"))
people_immunity_percent_info = people_immunity_percent_info.set_index('WeekDate')
people_immunity_percent_info

In [ ]:
plt.figure(figsize=(20,6))
plt.xticks(rotation=80)
plt.xlabel('Weeks') 
plt.ylabel('Percentage')
plt.title('Percentage of US people have immunity(Per Week)')
sns.lineplot(data=people_immunity_percent_info, palette="flare")

#### Question 6
<h4>Produce the same plot as in #5 for the states. You should have a function that takes the two
letter abbreviation for the state and produces the plot for the state. 

In [ ]:
# Number of People got at least one vaccination dose per state per day. 
people_had_covid_alive_state = immunityCountCal()
people_had_covid_alive_state['location'] = people_had_covid_alive_state['State'].map(abbrev_us_state)
people_with_one_vaccine_dose_state=us_state_vaccinations_data.filter(['date','location','people_vaccinated'],axis=1)
people_with_one_vaccine_dose_state=people_with_one_vaccine_dose_state[people_with_one_vaccine_dose_state['location'].isin(state_names_list)].reset_index(drop=True)
people_with_one_vaccine_dose_state

In [ ]:
# Number of People had covid and alive and people who got at least one vaccination dose per state per week. 
state_people_immunity_info=pd.merge(people_had_covid_alive_state,people_with_one_vaccine_dose_state, on=['date','location'],how='left').fillna(0)
state_people_immunity_info['date']= pd.to_datetime(state_people_immunity_info['date'])
state_people_immunity_info=state_people_immunity_info.groupby('State').resample('W-mon', label='right', closed = 'right', on='date').apply(lambda x : x.iloc[-1])
state_people_immunity_info=state_people_immunity_info.drop(['State','date'],axis=1)
state_people_immunity_info=state_people_immunity_info.reset_index()
state_people_immunity_info=state_people_immunity_info.merge(state_population,left_on='location',right_on='State_Names',how='left').reindex(columns=['State', 'date', 'Had Covid', 'location','people_vaccinated','population'])
state_people_immunity_info

In [ ]:
# Percentage of people have some immunity in each state per week
state_immune_percent=pd.DataFrame()
state_immune_percent=state_people_immunity_info.filter(['State','date'])                                                                             
state_immune_percent['% Had Covid'] = (state_people_immunity_info['Had Covid']/state_people_immunity_info['population'])*100
state_immune_percent['% Vaccinated'] =(state_people_immunity_info['people_vaccinated']/state_people_immunity_info['population'])*100
state_immune_percent['% Immune'] = ((state_people_immunity_info['Had Covid']+state_people_immunity_info['people_vaccinated'])/state_people_immunity_info['population'])*100
state_immune_percent['date']=state_immune_percent['date'].map(lambda tf : tf.strftime("%d-%m-%Y"))
state_immune_percent

In [ ]:
def statePlot(state_abbr):
    state_immune_percent_plot=state_immune_percent.copy()
    state_immune_percent_plot=state_immune_percent_plot[state_immune_percent_plot['State'].isin(state_abbr)]
    state_immune_percent_plot=state_immune_percent_plot.reset_index(drop=True)
    state_immune_percent_plot=state_immune_percent_plot.set_index('date')
    plt.figure(figsize=(20,6))
    plt.xticks(rotation=80)
    plt.xlabel('Weeks') 
    plt.ylabel('Percentage') 
    plt.title('Percentage of people have immunity for State(Per Week)')
    sns.lineplot(data=state_immune_percent_plot, palette="flare")
        
abbr=['AK']
statePlot(abbr)

#### Question 1
<h4>In us_state_vaccinations.csv there are some gaps in the data. Will those gaps affect the
results of the following questions. If no why not. If yes explain the affect.
    
        * Gaps will affect the question solved above.
        * As it is real time data the rows gets updated daily, I need latest data to compute question 2 and 3.
        * If the row updated have gap or nan which will affect my comuptaion result.
        * For question 4, 5 if there will be gaps then plot will not display the immune people % for half of the data
        * To avoid all this usiing forwardfill method and filling the gaps.